In [1]:
import requests
import json
import socket
import random
import time
from credentials import BEARER_TOKEN

class TweetListener:
    """ server to a stream of tweets
    """
    def __init__(self, Bearer_token, rules, tweet_params):
        self.bearer_oauth = self.get_bearer_oauth(Bearer_token)
        self.rules = rules
        self.tweet_params = tweet_params

    def get_bearer_oauth(self, Bearer_token):
        def bearer_oauth(r):
            """
            Method required by bearer token authentication.
            """
            r.headers["Authorization"] = f"Bearer {Bearer_token}"
            r.headers["User-Agent"] = "v2FilteredStreamPython"
            return r

        return bearer_oauth

    def get_rules(self):
        """ returns current rules

        Returns:
            json: current rules
        """
        response = requests.get(
            "https://api.twitter.com/2/tweets/search/stream/rules", auth=self.bearer_oauth
        )
        if response.status_code != 200:
            raise Exception(
                "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
            )
        return response.json()

    def clear_rules(self):
        """ send POST request to delete all exiting rules
        """
        old_rules = self.get_rules()
        if old_rules is not None and "data" in old_rules:
            ids = list(map(lambda rule: rule["id"], old_rules["data"]))
            payload = {"delete": {"ids": ids}}
            response = requests.post(
                "https://api.twitter.com/2/tweets/search/stream/rules",
                auth=self.bearer_oauth,
                json=payload
            )
            if response.status_code != 200:
                raise Exception(
                    "Cannot delete rules (HTTP {}): {}".format(
                        response.status_code, response.text
                    )
                )

    def set_rules(self, rules, clear=True):
        """ set new rules

        Args:
            rules (list): list of rules to add
            clear (bool, optional): delete all current rules. Defaults to True.
        """
        if clear:
            self.clear_rules()
        payload = {"add": rules}
        response = requests.post(
            "https://api.twitter.com/2/tweets/search/stream/rules",
            auth=self.bearer_oauth,
            json=payload,
        )
        if response.status_code != 201:
            raise Exception(
                "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
            )

    def get_stream(self):
        """ returns a tweet stream

        Returns:
            stream: tweets stream
        """
        response = requests.get(
            "https://api.twitter.com/2/tweets/search/stream", 
            auth=self.bearer_oauth, 
            stream=True,
            params=self.tweet_params   
        )
        if response.status_code != 200:
            raise Exception(
                "Cannot get stream (HTTP {}): {}".format(
                    response.status_code, response.text
                )
            )

        return response

    def send_tweets(self, client_socket, debug_mode=False, print_tweets=False):
        """ send tweets to the client socket

        Args:
            client_socket (socket): the client socket
            debug_mode (bool, optional): send fake messages (not real tweets), can be used when debugging to not avoid pulling real tweets. Defaults to False.
        """
        if debug_mode:
            while True:
                for _ in range(random.randint(1, 3)):
                    if random.randint(0, 1):
                        client_socket.send('good\n'.encode('utf-8'))
                    else:
                        client_socket.send('bad\n'.encode('utf-8'))
                wait = random.randint(10, 50)  # wait between 1 and 5 seconds
                time.sleep(wait/10)
        else:
            self.set_rules(self.rules)
            stream = self.get_stream()
            for response_line in stream.iter_lines():
                if response_line:
                    try:
                        tweet = json.loads(response_line)
                        if print_tweets:
                            print(tweet)
                        message = self.tweet_to_message(tweet)
                        if message:
                            message = json.dumps(message)+'\n'
                            client_socket.send(message.encode('utf-8'))
                            wait = random.randint(1, 20)  # wait between 0.1 and 2 seconds
                            time.sleep(wait/10)
                    except BaseException as e:
                        print("Error on_data: %s" % str(e))

    def tweet_to_message(self,tweet):
        """ process a tweet and returns an object with only the desired fields 

        Args:
            tweet (json): a tweet received from the Twitter API

        Returns:
            dict: an object contains only the desired fields of the tweet (text, location)
        """
        text = tweet['data']['text']
        text = text.split('https')[0]
        tag = tweet['matching_rules'][0]['tag']
        location = None
        for user in tweet['includes']['users']:
            if 'location' in user:
                location = user['location']
                break
        if text and location and tag:
            message = {
                'text': text,
                'location': location,
                'tag':tag
            }
            return message
        else:
            return None

    def start_server(self, port, host="127.0.0.1", debug_mode=False, print_tweets=False):
        """ starts the server on the chosen port

        Args:
            port (int): the port
            host (str, optional): the host. Defaults to "127.0.0.1".
            debug_mode (bool, optional): run the server in debug mode (send fake messages (not real tweets), can be used when debugging to not avoid pulling real tweets). Defaults to False.
        """
        new_skt = socket.socket()      # initiate a socket object
        new_skt.bind((host, port))     # Binding host and port
        print(f"Listening on http://{host}:{port}")
        new_skt.listen(5)              #  waiting for client connection
        client_socket, client_addr = new_skt.accept()     # Establish connection with client. it returns first a socket object,c, and the address bound to the socket
        print("Received request from: " + str(client_addr))
        self.send_tweets(client_socket, debug_mode=debug_mode, print_tweets=print_tweets)


In [ ]:
my_rules = [
    {"value": "samsung lang:en", "tag": "samsung"},
    {"value": "apple lang:en", "tag": "apple"},
    {"value": "amazon lang:en", "tag": "amazon"},
    {"value": "facebook lang:en", "tag": "facebook"},
]     
  
tweet_params = {
    'expansions':'author_id,geo.place_id',
    "place.fields" : 'geo,name,contained_within,country,country_code,full_name',
    "user.fields": ["location"]
}

In [ ]:
server = TweetListener(BEARER_TOKEN, my_rules, tweet_params)
server.start_server(port=4040)